<a href="https://colab.research.google.com/github/elangbijak4/algoritma-dinda-new/blob/main/k_allay_new2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import networkx as nx

# Fungsi untuk menghitung nilai benefit antara dua negara
def benefit_function(policy_1, policy_2):
    # Contoh fungsi: kombinasi jarak dan kekuatan
    distance = np.linalg.norm(policy_1['position'] - policy_2['position'])
    power_diff = abs(policy_1['power'] - policy_2['power'])
    return -distance + 0.5 * power_diff

# Algoritma Optimasi Berbasis Pasangan
def pairwise_optimization(countries, K, threshold=0.5):
    N = len(countries)
    G = nx.Graph()
    G.add_nodes_from([c['id'] for c in countries])

    # Buat matriks benefit dan bangun graf
    for i, country_1 in enumerate(countries):
        for j, country_2 in enumerate(countries):
            if i >= j:
                continue
            benefit_1 = benefit_function(country_1, country_2)
            benefit_2 = benefit_function(country_2, country_1)
            if benefit_1 + benefit_2 > threshold:
                G.add_edge(country_1['id'], country_2['id'])

    # Temukan komponen terhubung sebagai cluster awal
    initial_clusters = list(nx.connected_components(G))

    # Jika cluster terlalu banyak atau terlalu sedikit, optimasi
    while len(initial_clusters) > K:
        # Gabungkan dua cluster dengan nilai rata-rata minimum
        distances = []
        for i, c1 in enumerate(initial_clusters):
            for j, c2 in enumerate(initial_clusters):
                if i >= j:
                    continue
                avg_benefit = np.mean([
                    benefit_function(countries[n1 - 1], countries[n2 - 1])
                    for n1 in c1 for n2 in c2
                ])
                distances.append((avg_benefit, i, j))

        # Gabung dua cluster dengan nilai benefit tertinggi
        _, i, j = max(distances)
        initial_clusters[i].update(initial_clusters[j])
        del initial_clusters[j]

    while len(initial_clusters) < K:
        # Tambahkan negara baru ke cluster terkuat
        unclustered = set(G.nodes) - set.union(*initial_clusters)
        if not unclustered:
            break
        for cluster in initial_clusters:
            avg_benefit = np.mean([
                benefit_function(countries[n1 - 1], countries[n2 - 1])
                for n1 in cluster for n2 in unclustered
            ])
            selected = max(unclustered, key=lambda n: avg_benefit)
            cluster.add(selected)
            unclustered.remove(selected)

    return initial_clusters

# Contoh Data Negara
countries = [
    {'id': 1, 'position': np.array([1, 2]), 'power': 5},
    {'id': 2, 'position': np.array([2, 3]), 'power': 4},
    {'id': 3, 'position': np.array([3, 1]), 'power': 6},
    {'id': 4, 'position': np.array([6, 7]), 'power': 8},
    {'id': 5, 'position': np.array([7, 8]), 'power': 7},
]

# Jalankan Algoritma
clusters = pairwise_optimization(countries, K=2)
print("Hasil Clustering Optimasi Berbasis Pasangan:")
for cluster in clusters:
    print(f"Cluster: {list(cluster)}")

Hasil Clustering Optimasi Berbasis Pasangan:
Cluster: [1, 2, 3]
Cluster: [4, 5]


In [2]:
import numpy as np
import networkx as nx

# Fungsi untuk menghitung nilai benefit antara dua negara
def benefit_function(policy_1, policy_2):
    distance = np.linalg.norm(policy_1['position'] - policy_2['position'])
    power_diff = abs(policy_1['power'] - policy_2['power'])
    return -distance + 0.5 * power_diff

# Algoritma Optimasi Berbasis Pasangan
def pairwise_optimization(countries, K, threshold=0.5):
    N = len(countries)
    G = nx.Graph()
    G.add_nodes_from([c['id'] for c in countries])
    reasons = []

    # Buat matriks benefit dan bangun graf
    for i, country_1 in enumerate(countries):
        for j, country_2 in enumerate(countries):
            if i >= j:
                continue
            benefit_1 = benefit_function(country_1, country_2)
            benefit_2 = benefit_function(country_2, country_1)
            if benefit_1 + benefit_2 > threshold:
                G.add_edge(country_1['id'], country_2['id'])
                reasons.append({
                    'from': country_1['id'],
                    'to': country_2['id'],
                    'reason': f"Mutual Benefit = {benefit_1 + benefit_2:.2f}"
                })

    # Temukan komponen terhubung sebagai cluster awal
    initial_clusters = list(nx.connected_components(G))

    # Optimasi jika jumlah cluster tidak sesuai
    while len(initial_clusters) > K:
        distances = []
        for i, c1 in enumerate(initial_clusters):
            for j, c2 in enumerate(initial_clusters):
                if i >= j:
                    continue
                avg_benefit = np.mean([
                    benefit_function(countries[n1 - 1], countries[n2 - 1])
                    for n1 in c1 for n2 in c2
                ])
                distances.append((avg_benefit, i, j))
        _, i, j = max(distances)
        initial_clusters[i].update(initial_clusters[j])
        del initial_clusters[j]

    return initial_clusters, reasons

# Contoh Data Negara
countries = [
    {'id': 1, 'position': np.array([1, 2]), 'power': 5},
    {'id': 2, 'position': np.array([2, 3]), 'power': 4},
    {'id': 3, 'position': np.array([3, 1]), 'power': 6},
    {'id': 4, 'position': np.array([6, 7]), 'power': 8},
    {'id': 5, 'position': np.array([7, 8]), 'power': 7},
]

# Jalankan Algoritma
clusters, reasons = pairwise_optimization(countries, K=2)
print("\nHasil Clustering Optimasi Berbasis Pasangan:")
for cluster in clusters:
    print(f"\nCluster: {list(cluster)}")
for reason in reasons:
    print(f"Negara {reason['from']} dan {reason['to']} menjadi sekutu karena: {reason['reason']}")


Hasil Clustering Optimasi Berbasis Pasangan:

Cluster: [1, 2, 3]

Cluster: [4, 5]
